In [32]:
import json
import os
from pathlib import Path
import shutil

import numpy as np
import requests
import skimage.io

%cd /root/hidebound/python
from hidebound.core.database import Database

def setup_files():
    hb_root = Path('/tmp/hidebound')
    meta = Path(hb_root, 'metadata')
    if meta.exists():
        shutil.rmtree(meta)
        
    meta = Path(hb_root, 'data')
    if meta.exists():
        shutil.rmtree(meta)
        
    config = dict(
        exclude_regex='\.DS_Store',
        hidebound_directory='/tmp/hidebound',
        include_regex='',
        root_directory='/tmp/projects',
        specification_files=['/root/hidebound/python/hidebound/core/test_specifications.py'],
        write_mode='copy',
    )
    config_path = Path(hb_root, 'hidebound_config.json')
    with open(config_path, 'w') as f:
        json.dump(config, f)

    root = Path('/tmp/projects')
    if Path(root).exists():
        shutil.rmtree(root)

    filepaths = [
        'p-cat001/spec001/p-cat001_s-spec001_d-running-cat_v001/p-cat001_s-spec001_d-running-cat_v001_c000-005_f0001.png',
#         'p-cat001/spec001/p-cat001_s-spec001_d-running-cat_v001/p-cat001_s-spec001_d-Running-Cat_v001_c000-005_f0002.png',
        'p-cat001/spec001/p-cat001_s-spec001_d-running-cat_v001/p-cat001_s-spec001_d-running-cat_v001_c000-005_f0002.png',
        'p-cat001/spec001/p-cat001_s-spec001_d-running-cat_v001/p-cat001_s-spec001_d-running-cat_v001_c000-005_f0003.png',

        'p-cat001/s-spec002/d-tabby-playing/p-cat001_s-spec002_d-tabby-playing_v001/p-cat001_s-spec002_d-tabby-playing_v001_f0001.jpg',
        'p-cat001/s-spec002/d-tabby-playing/p-cat001_s-spec002_d-tabby-playing_v001/p-cat001_s-spec002_d-tabby-playing_v001_f0002.jpg',
        'p-cat001/s-spec002/d-tabby-playing/p-cat001_s-spec002_d-tabby-playing_v001/p-cat001_s-spec002_d-tabby-playing_v001_f0003.jpg',

        'p-cat001/s-spec002/d-tabby-playing/p-cat001_s-spec002_d-tabby-playing_v002/p-cat001_s-spec002_d-tabby-playing_v002_f0001.jpg',
        'p-cat001/s-spec002/d-tabby-playing/p-cat001_s-spec002_d-tabby-playing_v002/p-cat001_s-spec002_d-tabby-playing_v002_f0002.jpg',
        'p-cat001/s-spec002/d-tabby-playing/p-cat001_s-spec002_d-tabby-playing_v002/p-cat001_s-spec002_d-tabby-playing_v002_f0003.jpg',

        'p-cat001/s-spec002/d-calico-jumping/p-cat001_s-spec002_d-calico-jumping_v001/p-cat001_s-spec002_d-calico-jumping_v001_f0001.jpg',
        'p-cat001/s-spec002/d-calico-jumping/p-cat001_s-spec002_d-calico-jumping_v001/p-cat001_s-spec002_d-calico-jumping_v001_f0002.jpg',
        'p-cat001/s-spec002/d-calico-jumping/p-cat001_s-spec002_d-calico-jumping_v001/p-cat001_s-spec002_d-calico-jumping_v001_f0003.jpg',
    ]
    filepaths = [Path(root, x) for x in filepaths]

    for filepath in filepaths:
        os.makedirs(filepath.parent, exist_ok=True)

        img = np.zeros((1024, 1024, 3), dtype=np.uint8)
        skimage.io.imsave(filepath.as_posix(), img)

setup_files()
!tree /tmp/hidebound; tree /tmp/projects

/root/hidebound/python


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:63: UserWarning: /tmp/projects/p-cat001/spec001/p-cat001_s-spec001_d-running-cat_v001/p-cat001_s-spec001_d-running-cat_v001_c000-005_f0001.png is a low contrast image
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:63: UserWarning: /tmp/projects/p-cat001/spec001/p-cat001_s-spec001_d-running-cat_v001/p-cat001_s-spec001_d-running-cat_v001_c000-005_f0002.png is a low contrast image
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:63: UserWarning: /tmp/projects/p-cat001/spec001/p-cat001_s-spec001_d-running-cat_v001/p-cat001_s-spec001_d-running-cat_v001_c000-005_f0003.png is a low contrast image
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:63: UserWarning: /tmp/projects/p-cat001/s-spec002/d-tabby-playing/p-cat001_s-spec002_d-tabby-playing_v001/p-cat001_s-spec002_d-tabby-playing_v001_f0001.jpg is a low contrast image
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:63: UserWarnin

/tmp/hidebound
├── hidebound_config.json
└── specifications

1 directory, 1 file
/tmp/projects
└── p-cat001
    ├── s-spec002
    │   ├── d-calico-jumping
    │   │   └── p-cat001_s-spec002_d-calico-jumping_v001
    │   │       ├── p-cat001_s-spec002_d-calico-jumping_v001_f0001.jpg
    │   │       ├── p-cat001_s-spec002_d-calico-jumping_v001_f0002.jpg
    │   │       └── p-cat001_s-spec002_d-calico-jumping_v001_f0003.jpg
    │   └── d-tabby-playing
    │       ├── p-cat001_s-spec002_d-tabby-playing_v001
    │       │   ├── p-cat001_s-spec002_d-tabby-playing_v001_f0001.jpg
    │       │   ├── p-cat001_s-spec002_d-tabby-playing_v001_f0002.jpg
    │       │   └── p-cat001_s-spec002_d-tabby-playing_v001_f0003.jpg
    │       └── p-cat001_s-spec002_d-tabby-playing_v002
    │           ├── p-cat001_s-spec002_d-tabby-playing_v002_f0001.jpg
    │           ├── p-cat001_s-spec002_d-tabby-playing_v002_f0002.jpg
    │           └── p-cat001_s-spec002_d-tabby-playing_v002_f0003.jpg
    └── spec001

In [33]:
url = 'http://0.0.0.0:5000/api/initialize'
config = dict(
    root_directory='/tmp/projects',
    hidebound_directory='/tmp/hidebound',
    specification_files=['/root/hidebound/python/hidebound/core/test_specifications.py']
)
config = json.dumps(config)
response = requests.post(url, json=config).json()['message']
print(response)

url = 'http://0.0.0.0:5000/api/update'
response = requests.post(url).json()['message']
print(response)

# url = 'http://0.0.0.0:5000/api/read'
# response = requests.post(url).json()['response']

# url = 'http://0.0.0.0:5000/api/search'
# params = json.dumps({'query': 'SELECT * FROM data WHERE asset_valid'})
# response = requests.post(url, json=params).json()['response']

url = 'http://0.0.0.0:5000/api/create'
response = requests.post(url).json()['message']
print(response)

!tree /tmp/hidebound; tree /tmp/projects

Database initialized.
Database updated.
Hidebound data created.
/tmp/hidebound
├── data
│   └── p-cat001
│       ├── s-spec002
│       │   ├── d-calico-jumping
│       │   │   └── p-cat001_s-spec002_d-calico-jumping_v001
│       │   │       ├── p-cat001_s-spec002_d-calico-jumping_v001_f0001.jpg
│       │   │       ├── p-cat001_s-spec002_d-calico-jumping_v001_f0002.jpg
│       │   │       └── p-cat001_s-spec002_d-calico-jumping_v001_f0003.jpg
│       │   └── d-tabby-playing
│       │       ├── p-cat001_s-spec002_d-tabby-playing_v001
│       │       │   ├── p-cat001_s-spec002_d-tabby-playing_v001_f0001.jpg
│       │       │   ├── p-cat001_s-spec002_d-tabby-playing_v001_f0002.jpg
│       │       │   └── p-cat001_s-spec002_d-tabby-playing_v001_f0003.jpg
│       │       └── p-cat001_s-spec002_d-tabby-playing_v002
│       │           ├── p-cat001_s-spec002_d-tabby-playing_v002_f0001.jpg
│       │           ├── p-cat001_s-spec002_d-tabby-playing_v002_f0002.jpg
│       │           └── p-cat001